# 最終課題
PLSAを使って文書集合をモデリングしトピック分析をおこなう。

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.12-1ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
mecab is already installed at the requested version (0.996-10build1)
libmecab-dev is already installed at the requested version (0.996-10build1)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-2.1)
git is already installed at the requested version (1:2.25.1-1ubuntu3.8)
make is already installed at the requested version (4.2.1-1.2)
curl is already installed at the requested version (7.68.0-1ubuntu2.15)
xz-utils is already installed at the requested version (5.2.4-1ubuntu1.1)
file is already installed at the requested version (1:5.38-4)
mecab is already installed at the requested version (0.996-10build1)
libmecab-dev is already installed at the requested version (0.996-10build1)
mecab-ipadic-utf8 is already installed at the requested ver

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import drive
import pandas as pd
import os
import re
import MeCab

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# データ準備

## データロード
https://www.rondhuit.com/download.html
からダンロードした「ldcc-20140209.tar」を使う。

今回sports-watch、it-life-hack、movie-enter３種類のデータを使う

In [ ]:
path_sports = '/content/drive/My Drive/Statistics/text/sports-watch'
path_it = '/content/drive/My Drive/Statistics/text/it-life-hack'
path_movie = '/content/drive/My Drive/Statistics/text/movie-enter'

In [ ]:
# 正規表現
def regular(data):
  # タイトル前の文字を削除
  target = '】'
  idx = data.find(target)
  data = data[idx+1:]

  # URLを削除
  data = re.sub(r"http://[a-zA-Z0-9.?/&=:]*", "", data)

  # 数値と英語文字を削除
  data = re.sub(r"[0-9a-zA-Z]", "", data)

  # 特殊記号などを削除
  data = re.sub(r"[:|「|!|＆|」｜\n｜\u3000|(|)|-|,|、|*|+|〜|\"|#|『|』|-|$|%|&|.|?|（|——|。|\/]|@|;|^|']", "", data)

  return data

In [ ]:
# データを読み込み
def getDataset(path_list):
  textList = []

  for path in path_list:
    # フォルダをtraverse 
    files= os.listdir(path)

    for file in files:
      # txtファイルpathを取得
      position = path + '/' + file

      with open(position, "r",encoding='utf-8') as f:
        data = f.read()

        # 不要な文字を削除
        data = regular(data)

        textList.append(data)

  return textList

In [ ]:
path_list = [path_sports, path_it, path_movie]

textList = getDataset(path_list)

In [ ]:
# 文章数
len(textList)

2640

In [ ]:
textList[0]

'浅田姉妹が徹子の洗礼祝年突入記念徹子の部屋スペシャル日放送）ではフィギュアスケートの浅田真央舞が姉妹で出演した収録は真央が世界選手権から帰国した直後の月日に行われたことからちょっと休みたいんだけどみたいな時はないですか？と尋ねる黒柳これに真央が今日はすごい楽しみですと答えるやホントにーと一気にテンションが上がりだし今日はちょっと滑って頂けるってうっふっふと実際のリンクで真央に演技をねだると普通に滑ってねあのもうそんなトリプルアクセルとかいいですからと続け徹子節を全開にしたまた人で食事をしながら行われたフリートークでは真央が舞から貰ったという熱田神宮の勝守を常に携帯していることに話しが及ぶすると徹子はなんかすごく真央ちゃんが落ち込んでスケート辞めたいってと言いかけ話を核心に進めようとした瞬間運ばれてきた料理にこれ何ですか？と突然話題を転換徹子の洗礼を受けた浅田姉妹はただ笑うしかなかった'

## 語彙変換

In [ ]:
tagger = MeCab.Tagger('-Ochasen')

# 日本語(名詞、形容詞、動詞)を抽出
def japanese_analyzer(string):
  result_list = []

  tagger = MeCab.Tagger("-Ochasen")
  tagger.parse("")
  node = tagger.parseToNode(string)

  while node:
    if node.feature.split(",")[0] == u"名詞":
      result_list.append(node.surface)
    elif node.feature.split(",")[0] == u"形容詞":
      result_list.append(node.feature.split(",")[6])
    elif node.feature.split(",")[0] == u"動詞":
      result_list.append(node.feature.split(",")[6])
        
    node = node.next
  
  return result_list

In [ ]:
# テキスト特徴量をカウント
count_vect = CountVectorizer(min_df=0.008, max_df=0.05, analyzer=japanese_analyzer)
textList_trans = count_vect.fit_transform(textList)

In [ ]:
# 語彙数
len(count_vect.vocabulary_)

2494

In [ ]:
count_vect.get_feature_names()[50:60]

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['お話', 'お金', 'お願い', 'かかる', 'かなう', 'かなり', 'かねる', 'かわいい', 'がち', 'がる']

In [ ]:
textList_trans = textList_trans.toarray()
textList_trans.shape

(2640, 2494)

In [ ]:
# 変換後の単語
dataList = []
for index, word in enumerate(textList_trans[0]):
  if word == 1:
    dataList.append(count_vect.get_feature_names_out()[index])
print(dataList)

['だす', 'ちゃん', 'スペシャル', 'トーク', 'フィギュア', 'ホント', 'ー', '上がる', '及ぶ', '収録', '尋ねる', '帰国', '携帯', '料理', '普通', '楽しみ', '演技', '直後', '瞬間', '笑う', '節', '貰う', '辞める', '進める', '運ぶ', '選手権', '部屋', '頂ける', '食事']


# モデル定義

In [ ]:
D, W = textList_trans.shape
K = 6
print(f"{D} documents, {W} different words, and {K} topics")

2640 documents, 2494 different words, and 6 topics


## M step
* モデルパラメータを更新する。

In [ ]:
def m_step(counts, q):
  pseudo_counts = counts[:, :, None] * q
  theta = pseudo_counts.sum(1)
  # ゼロで割った箇所がnanになっていますので、判断処理を追加
  theta = np.divide(theta, theta.sum(-1, keepdims=True), 
                    out=np.zeros_like(theta), where=theta.sum(-1, keepdims=True)!=0)
  
  phi = pseudo_counts.sum(0)
  phi = np.divide(phi, phi.sum(0, keepdims=True),
                  out=np.zeros_like(phi), where=phi.sum(0, keepdims=True)!=0)
  
  return theta, phi

## E step

In [ ]:
def e_step(theta, phi):
  q = theta[:, None, :] * phi[None, :, :]
  q = np.divide(q, q.sum(-1, keepdims=True),
                out=np.zeros_like(q), where=q.sum(-1, keepdims=True)!=0)
  return q

## lower boundの計算
* EMアルゴリズムがlower boundを大きくしていっているかチェックするため。

In [ ]:
def lower_bound(counts, q, theta, phi):
  pseudo_counts = counts[:, :, None] * q
  lb = (pseudo_counts * np.log(theta[:, None, :] + 1e-16)).sum()
  lb += (pseudo_counts * np.log(phi[None, :, :] + 1e-16)).sum()
  lb -= (pseudo_counts * np.log(q + 1e-16)).sum()
  return lb

## EMアルゴリズムの実行

In [ ]:
q = np.random.randn(D, W, K)
q = np.exp(q) / np.exp(q).sum(-1, keepdims=True)

In [ ]:
for i in range(50):
  theta, phi = m_step(textList_trans, q)
  q = e_step(theta, phi)
  lb = lower_bound(textList_trans, q, theta, phi)
  print(f"iter {i+1} | lower bound {lb:.4e}")

iter 1 | lower bound -1.4423e+06
iter 2 | lower bound -1.4420e+06
iter 3 | lower bound -1.4414e+06
iter 4 | lower bound -1.4405e+06
iter 5 | lower bound -1.4388e+06
iter 6 | lower bound -1.4356e+06
iter 7 | lower bound -1.4299e+06
iter 8 | lower bound -1.4206e+06
iter 9 | lower bound -1.4071e+06
iter 10 | lower bound -1.3912e+06
iter 11 | lower bound -1.3761e+06
iter 12 | lower bound -1.3640e+06
iter 13 | lower bound -1.3549e+06
iter 14 | lower bound -1.3481e+06
iter 15 | lower bound -1.3429e+06
iter 16 | lower bound -1.3387e+06
iter 17 | lower bound -1.3354e+06
iter 18 | lower bound -1.3327e+06
iter 19 | lower bound -1.3302e+06
iter 20 | lower bound -1.3281e+06
iter 21 | lower bound -1.3263e+06
iter 22 | lower bound -1.3247e+06
iter 23 | lower bound -1.3233e+06
iter 24 | lower bound -1.3221e+06
iter 25 | lower bound -1.3211e+06
iter 26 | lower bound -1.3201e+06
iter 27 | lower bound -1.3194e+06
iter 28 | lower bound -1.3187e+06
iter 29 | lower bound -1.3180e+06
iter 30 | lower bound -

# トピック語の表示
今回sports-watch、it-life-hack、movie-enter３種類のデータを使う

In [ ]:
topic_words = np.argsort(- phi, axis=0)
for k in range(K):
  print(k, end=' : ')
  for i in range(40):
    print(count_vect.get_feature_names_out()[topic_words[i, k]], end=', ')
  print()

0 : 祭, 応募, キャンペーン, 人類, まとめ, 当選, 試写, 招待, プレゼント, アカデミー, いただく, 今週, 謎, 動員, 事件, 受賞, 国際, 連絡, 部門, 弾, プレミア, 億, ダーク, ポスター, 上映, ナイト, 女, プロメテウス, 組, 神, 週, 犯罪, 券, 水, 闇, ベルセルク, 衝撃, ヶ, いたす, 日月, 
1 : 孫, 社長, 虎の巻, 友達, ロゴ, 起動, 作成, 登録, 投稿, 表, 検索, キー, ワザ, リスト, 開く, 共有, 術, マーク, ブック, 指定, 管理, アクセス, 列, ツール, 正義, マウス, セキュリティ, ダウンロード, 保存, 付ける, 快適, タグ, メール, タブ, ン, 行, コピー, ドライブ, グループ, メニュー, 
2 : 笑, 紺, ちゃん, ジョン, リアル, キャラクター, カーター, ヒーロー, 曲, 銃, 絆, 恐怖, 原作, 主題歌, 士, 俳優, 彼ら, ロボット, あなた, 素晴らしい, ふる, 妻, 会場, 息子, 恋愛, 女, ヴァンパイア, 無い, ぉっくす, 生, 恋, 男性, スティール, 生きる, 観, ブラック, 出来る, 父, ディズニー, 共演, 
3 : なでしこ, 芸能, 週刊, 撮る, 澤, アサヒ, ツイッター, ％, 星, 展, ツイート, 調査, 香川, 移籍, 同誌, 批判, 中国, 韓国, 記者, 吉田, 紙, 号, ２, 億, 空間, １, 本田, 活動, 件, 率, 更新, 契約, 希, 新聞, アジア, 球団, 所属, 展示, 商品, 掲示板, 
4 : 容量, バッテリー, ケース, 書籍, 音声, ノート, 接続, 電子, タブレット, 端末, 認識, 充電, スマホ, ダウンロード, ドコモ, 通信, 録画, 液晶, ソフト, 本体, 電源, ケーブル, 音楽, サイズ, 装着, レビュー, 英語, 募集, 無線, キーボード, レ, キヤノン, インテル, 携帯, 内蔵, ビューアー, ディスプレイ, 解像度, 機種, 市場, 
5 : 野村, 町, 斎藤, 会議, ボール, プレー, 佑, 田中, 開幕, 巨人, 楽天, シーズン, 長友, 訊く, 投げる, 挙げる, 予選, フジテレビ, 本田, 